# Starter data joining/cleaning code (SK, 23 11 28)

have tended to save a csv copy of every dataframe created, can take these out once happy with the code

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
# pip install sqlalchemy

In [ ]:
# Load main csv with salary data (local currency & gbp converted)
sal_orig_df = pd.read_csv("output_playaround231128.csv")

Bring in data from country_codes table, to merge with salaries data (NEED the country name, the country code is difficult to work with!)

In [ ]:
# mysql.connector or pymysql don't work with Jupyter / MySQl

In [ ]:
# country_codes.sql information is in an .sql table. Need to convert sql > db in order for pandas to turn into into DF. Then convert to csv
from sqlalchemy import create_engine

# MySQl database connection details
username = 'root'
password = '' # complete this
host = 'localhost'
database = 'countries_db'

# Creates a database engine
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}")

# The SQL query to get all info from table named country_codes
query = "SELECT * FROM country_codes"  

# Use Pandas to load data into a DataFrame
countries_df = pd.read_sql_query(query, engine)

In [ ]:
print(countries_df.head())

In [ ]:
countries_df.to_csv("country_codes.csv", encoding='utf-8', index=False) # save dataframe to .csv file

In [ ]:
# Outer join salaries df to country_codes df on iso_alpha2
sal_country_df = pd.merge(sal_orig_df, countries_df, on='iso_alpha2', how='outer')
sal_country_df.to_csv("sal_and_country.csv", index=False) # save dataframe to .csv file

In [ ]:
sal_country_df.describe()

In [ ]:
sal_country_df.head()

### Cleaning of merge salaries & country (codes) info

Remove the excess rows from country_codes which aren't in our Teleport data. Identify the excess rows... 

In [ ]:
sal_country_df['salary_percentiles_percentile_50'].isnull().sum()
# 53 country rows from country_codes don't have salary data (so weren't in Teleport API of 198 / 252 countries )

Want to grab the country codes which aren't needed (no salary data), put them in a DF, and export to a csv to keep a record, then delete them from the main df...

In [ ]:
# Filter rows where 'salary_percentiles_percentile_50' is null
countries_no_salaries_df = sal_country_df[sal_country_df['salary_percentiles_percentile_50'].isnull()]

# Export the filtered DataFrame to a CSV file
countries_no_salaries_df.to_csv('countries_no_salaries.csv', index=False) # should contain 53

# Drop the filtered rows from the original DataFrame
sal_country_df = sal_country_df.dropna(subset=['salary_percentiles_percentile_50'])

### Merge gender pay column in with the main salary_country df

In [ ]:
# Read in the gender_pay_parity csv column
gender_df = pd.read_csv("Gender Pay Gap.csv")
gender_df

In [ ]:
# Cleaning: rename Country to country / Gender_Pay_Parity gender_pay_parity to facilitate merge
gender_df.rename(columns={'Country': 'country', 'Gender_Pay_Parity':'gender_pay_parity'}, inplace=True)
gender_df.head(10)

In [ ]:
# join main salary_country DF with gender_gap column
sal_country_gender_df = pd.merge(gender_df, sal_country_df, on='country', how='outer')
sal_country_gender_df.to_csv("sal_and_country_and_gender.csv", index=False) # save dataframe to .csv file
sal_country_gender_df.head(10) 

Need to look through the now triple-joined df and look for any odd rows where the country name from gender_pay_parity didn't match the spelling of the country name in salary_countries (it wasn't joined on iso_alpha2 code (it wasn't in gender_pay.csv), it was joined on name which is will have more variation).  
Options: clean the country names before the merge to make sure they match. Or, import from an SQL table (as this will have been properly processed).

Need to clean these names, if they are still in the dataset 
CuraÃ§ao	Willemstad
Ã…land	Mariehamn
Saint BarthÃ©lemy	Gustavia
RÃ©union	Saint-Denis
SÃ£o TomÃ© and PrÃ­ncipe

# Define the dodgy characters to search for
dodgy_character = 'Ã' AND .... 

# Create a boolean mask to identify rows with the dodgy character in any field
mask = df.apply(lambda x: x.str.contains(dodgy_character)).any(axis=1)

# Get the rows with the dodgy character
dodgy_rows = df[mask]

# Print or process the dodgy rows
print(dodgy_rows)


# Scraps

In [ ]:
#sal_country_df.isna().sum() # something wrong: it's saying no NaN values # 156

In [ ]:
data_types = sal_country_df.dtypes
data_types